# Case Study #4: Data Bank
The case study questions presented here are created by [**Data With Danny**](https://linktr.ee/datawithdanny). They are part of the [**8 Week SQL Challenge**](https://8weeksqlchallenge.com/).

My SQL queries are written in the `PostgreSQL 15` dialect, integrated into `Jupyter Notebook`, which allows us to instantly view the query results and document the queries.

For more details about the **Case Study #4**, click [**here**](https://8weeksqlchallenge.com/case-study-4/).

## Table of Contents

### [1. Importing Libraries](#Import)

### [2. Tables of the Database](#Tables)

### [3. Case Study Questions](#CaseStudyQuestions)

- [A. Customer Nodes Exploration](#A)
- [B. Customer Transactions](#B)
- [C. Data Allocation Challenge](#C)

___
<a id = 'Import'></a>
## 1. Importing required Libraries

In [1]:
import psycopg2 as pg2
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

___
<a id = 'Tables'></a>
## 2. Tables of the Database

### Connecting PostgreSQL database through Jupyter Notebook

In [2]:
# Get the PostgreSQL password 
mypassword = os.getenv("POSTGRESQL_PASSWORD")

# Connect SQL database
conn = pg2.connect(user = 'postgres', password = mypassword, database = 'data_bank')
cursor = conn.cursor()

Now, let's list the table names of the `data_bank` database.

In [3]:
query_ShowTables = """
SELECT 
    table_schema, 
    table_name
FROM information_schema.tables
WHERE table_schema = 'data_bank';
"""
cursor.execute(query_ShowTables)

print('--- Tables within "data_bank" database --- ')
for table in cursor:
    print(table[1])

--- Tables within "data_bank" database --- 
regions
customer_nodes
customer_transactions


The followings are the 3 tables within the `data_bank` database. Please click [**here**](https://8weeksqlchallenge.com/case-study-4/) to get more insights about the tables.

In [4]:
cursor.execute(query_ShowTables)
for table in cursor:
    print("Table: ", table[1])
    query = "SELECT * FROM " + table[0] + '.' + table[1]
    df = pd.read_sql(query, conn)
    display(df)

Table:  regions


,region_id,region_name
0,1,Australia
1,2,America
2,3,Africa
3,4,Asia
4,5,Europe


Table:  customer_nodes


,customer_id,region_id,node_id,start_date,end_date
0,1,3,4,2020-01-02,2020-01-03
1,2,3,5,2020-01-03,2020-01-17
2,3,5,4,2020-01-27,2020-02-18
3,4,5,4,2020-01-07,2020-01-19
4,5,3,3,2020-01-15,2020-01-23
...,...,...,...,...,...
3495,496,3,4,2020-02-25,9999-12-31
3496,497,5,4,2020-05-27,9999-12-31
3497,498,1,2,2020-04-05,9999-12-31
3498,499,5,1,2020-02-03,9999-12-31


Table:  customer_transactions


,customer_id,txn_date,txn_type,txn_amount
0,429,2020-01-21,deposit,82
1,155,2020-01-10,deposit,712
2,398,2020-01-01,deposit,196
3,255,2020-01-14,deposit,563
4,185,2020-01-29,deposit,626
...,...,...,...,...
5863,189,2020-02-03,withdrawal,870
5864,189,2020-03-22,purchase,718
5865,189,2020-02-06,purchase,393
5866,189,2020-01-22,deposit,302


___
<a id = 'CaseStudyQuestions'></a>
## Case Study Questions

<a id = 'A'></a>
## A. Customer Nodes Exploration

#### 1. How many unique nodes are there on the Data Bank system?

In [5]:
pd.read_sql("""
SELECT COUNT(DISTINCT node_id) AS nodes_count
FROM data_bank.customer_nodes
""", conn)

,nodes_count
0,5


**Result (Attempt #1)**\
In the Data Bank system, there are 5 unique nodes to which customers will be randomly reallocated.

However, if we interpret the question differently, we may understand that nodes are unique for each region, and customers are randomly distributed across the nodes *according to their region*. The following query shows the possible uniques nodes in each region.

**Here is the attempt #2**

In [6]:
pd.read_sql("""
SELECT 
    cn.region_id, 
    r.region_name, 
    STRING_AGG(DISTINCT cn.node_id::VARCHAR(1), ', ') AS nodes
FROM data_bank.customer_nodes cn
JOIN data_bank.regions r ON cn.region_id = r.region_id
GROUP BY cn.region_id, r.region_name
""", conn)

,region_id,region_name,nodes
0,1,Australia,"1, 2, 3, 4, 5"
1,2,America,"1, 2, 3, 4, 5"
2,3,Africa,"1, 2, 3, 4, 5"
3,4,Asia,"1, 2, 3, 4, 5"
4,5,Europe,"1, 2, 3, 4, 5"


In [7]:
pd.read_sql("""
SELECT SUM(nb_nodes)::INTEGER AS total_nodes
FROM
(
    -- Find the number of unique nodes per region
    
    SELECT region_id, COUNT(DISTINCT node_id) AS nb_nodes
    FROM data_bank.customer_nodes
    GROUP BY region_id
) n
""", conn)

,total_nodes
0,25


**Result (Attempt #2)**\
Hence, there are 25 unique nodes in the Data Bank system accross the world.

___
#### 2. What is the number of nodes per region?

In [8]:
pd.read_sql("""
SELECT 
    r.region_name, 
    COUNT(DISTINCT node_id) AS nb_nodes
FROM data_bank.customer_nodes cn
JOIN data_bank.regions r ON cn.region_id = r.region_id
GROUP BY r.region_name; 
""", conn)

,region_name,nb_nodes
0,Africa,5
1,America,5
2,Asia,5
3,Australia,5
4,Europe,5


**Result**\
There are 5 nodes per region.

___
#### 3. How many customers are allocated to each region?

In [9]:
pd.read_sql("""
SELECT 
    r.region_name AS region, 
    COUNT(DISTINCT customer_id) AS nb_customers
FROM data_bank.customer_nodes cn
JOIN data_bank.regions r ON cn.region_id = r.region_id
GROUP BY r.region_name
ORDER BY nb_customers DESC; 
""", conn)

,region,nb_customers
0,Australia,110
1,America,105
2,Africa,102
3,Asia,95
4,Europe,88


**Result**
- The majority of Data Bank customers are from Australia, totaling 110 customers.
- Following Australia, America and Africa are the second and third regions with the highest number of Data Bank customers.
- Asia has a total of 95 Data Bank customers.
- Data Bank does not seem to attract many customers from Europe, with only 88 clients.

___
#### 4. How many days on average are customers reallocated to a different node?

In [10]:
pd.read_sql("""
SELECT ROUND(AVG(end_date::date - start_date::date), 1) AS avg_reallocation_days
FROM data_bank.customer_nodes
WHERE end_date != '9999-12-31'
""", conn)

,avg_reallocation_days
0,14.6


**Result (Attempt #1)**\
Customers are reallocated to a different node on average after 14.6 days .

However, if we examine the `customer_id 7`, we can observe two instances where the customer has been reallocated to the same node (nodes 2 and 4). For customer ID 7, it took him/her 
- 21 days to be reallocated from node 4 to node 2
- 34 days to be reallocated from node 2 to node 4

In [11]:
pd.read_sql("""
SELECT * 
FROM data_bank.customer_nodes
WHERE customer_id = 7
""", conn)

,customer_id,region_id,node_id,start_date,end_date
0,7,2,5,2020-01-20,2020-02-04
1,7,2,4,2020-02-05,2020-02-20
2,7,2,4,2020-02-21,2020-02-26
3,7,2,2,2020-02-27,2020-03-05
4,7,2,2,2020-03-06,2020-04-01
5,7,2,4,2020-04-02,2020-04-07
6,7,2,5,2020-04-08,9999-12-31


**Table 1**: Customer 7 reallocated to node no 4 twice

node_id | start_date | end_date | nb_reallocation_days
--- | --- | --- | ---
4 | 2020-02-05 | 2020-02-20	| --
4 |	2020-02-21 | 2020-02-26	| 21

**Table 2**: Customer 7 reallocated to node no 2 twice 

node_id | start_date | end_date | nb_reallocation_days
--- | --- | --- | ---
2 | 2020-02-27 | 2020-03-05 | --
2 | 2020-03-06 | 2020-04-01	| 34

Hence, here is a query for the `reallocation_cte` that will be used later to answer the question. The query takes into consideration counting the total number of days for which customers will be reallocated to a different node, and includes all instances of both same and different nodes.

In [12]:
# Let's see for customer ID 7

pd.read_sql("""
SELECT *,
    CASE 
        WHEN LEAD(node_id) OVER w = node_id THEN NULL 
        WHEN LAG(node_id) OVER w = node_id THEN end_date::date - LAG(start_date) OVER w::date
        ELSE end_date::date - start_date::date
    END AS nb_days
FROM data_bank.customer_nodes
WHERE end_date != '9999-12-31' AND customer_id = 7
WINDOW w AS (PARTITION BY customer_id ORDER BY start_date)
""", conn)

,customer_id,region_id,node_id,start_date,end_date,nb_days
0,7,2,5,2020-01-20,2020-02-04,15.0
1,7,2,4,2020-02-05,2020-02-20,NaN
2,7,2,4,2020-02-21,2020-02-26,21.0
3,7,2,2,2020-02-27,2020-03-05,NaN
4,7,2,2,2020-03-06,2020-04-01,34.0
5,7,2,4,2020-04-02,2020-04-07,5.0


Now, let's utilize the aforementioned query as the `reallocation_cte`, to provide a comprehensive answer.

In [13]:
pd.read_sql("""
WITH reallocation_cte AS
(
    -- Find the number of reallocation days
    SELECT *, 
        CASE 
            WHEN LEAD(node_id) OVER w = node_id THEN NULL 
            WHEN LAG(node_id) OVER w = node_id THEN end_date::date - LAG(start_date) OVER w::date
            ELSE end_date::date - start_date::date
        END AS nb_reallocation_days
    FROM data_bank.customer_nodes
    WHERE end_date != '9999-12-31' 
    WINDOW w AS (PARTITION BY customer_id ORDER BY start_date)
)
SELECT ROUND(AVG(nb_reallocation_days),1) AS avg_reallocation_days
FROM reallocation_cte;
""", conn)

,avg_reallocation_days
0,17.3


**Result (Attempt #2)**\
Customers are reallocated to a different node on average after 17.3 days .

___
#### 5. What is the median, 80th and 95th percentile for this same reallocation days metric for each region?

Since I have previously addressed the reallocation days problem (see question #4 above) using two different approaches, I will also provide my answer to this question in two attempts. Please note that the questions 4 and 5 are rather vague.

**Result (Attempt #1)**

In [14]:
pd.read_sql("""
SELECT
    region_name, 
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY reallocation_days) as median,
    PERCENTILE_DISC(0.8) WITHIN GROUP(ORDER BY reallocation_days) as percentile_80th,
    PERCENTILE_DISC(0.95) WITHIN GROUP(ORDER BY reallocation_days) as percentile_95th
FROM
(
    SELECT 
        n.node_id, 
        n.start_date, 
        n.end_date, 
        r.region_name, 
        n.end_date::date - n.start_date::date AS reallocation_days
    FROM data_bank.customer_nodes n
    JOIN data_bank.regions r ON n.region_id = r.region_id
    WHERE n.end_date != '9999-12-31' 
) re
GROUP BY region_name;
""", conn)

,region_name,median,percentile_80th,percentile_95th
0,Africa,15,24,28
1,America,15,23,28
2,Asia,15,23,28
3,Australia,15,23,28
4,Europe,15,24,28


**Result (Attempt #2)**

In [15]:
pd.read_sql("""
WITH reallocation_cte AS
(
    SELECT *, 
        CASE 
            WHEN LEAD(node_id) OVER w = node_id THEN NULL 
            WHEN LAG(node_id) OVER w = node_id THEN end_date::date - LAG(start_date) OVER w::date
            ELSE end_date::date - start_date::date
        END AS nb_reallocation_days
    FROM data_bank.customer_nodes
    WHERE end_date != '9999-12-31' 
    WINDOW w AS (PARTITION BY customer_id ORDER BY start_date)
)
SELECT 
    r.region_name, 
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY cte.nb_reallocation_days) as median,
    PERCENTILE_DISC(0.8) WITHIN GROUP(ORDER BY cte.nb_reallocation_days) as percentile_80th,
    PERCENTILE_DISC(0.95) WITHIN GROUP(ORDER BY cte.nb_reallocation_days) as percentile_95th
FROM reallocation_cte cte
JOIN data_bank.regions r ON cte.region_id = r.region_id
GROUP BY r.region_name;
""", conn)

,region_name,median,percentile_80th,percentile_95th
0,Africa,17,27,36
1,America,17,26,36
2,Asia,17,25,34
3,Australia,17,26,36
4,Europe,18,27,37


___
<a id = 'B'></a>
## B. Customer Transactions
#### 1. What is the unique count and total amount for each transaction type?

In [16]:
pd.read_sql("""
SELECT 
    txn_type AS transaction_type, 
    to_char(COUNT(txn_type), 'FM 999,999') AS count, 
    to_char(SUM(txn_amount), 'FM$ 999,999,999.99') AS total_amount
FROM data_bank.customer_transactions
GROUP BY txn_type
""", conn)

,transaction_type,count,total_amount
0,purchase,"1,617","$ 806,537."
1,withdrawal,"1,580","$ 793,003."
2,deposit,"2,671","$ 1,359,168."


**Result**
- The most fequently used transaction type at Data Bank is **deposit**, with a total numbers of 2,671 deposits, amounting to $1,359,168.
- There is a total of 1,671 **purchases** made at Data Bank, totalling \\$ 806,537.
- There is a total of 1,580 **withdrawals** made at Data Bank, totalling \\$ 793,003

___
#### 2. What is the average total historical deposit counts and amounts for all customers?

In [17]:
pd.read_sql("""
SELECT 
    AVG(deposit_count)::INTEGER As nb_deposit, 
    to_char(AVG(avg_deposit_amount), 'FM$ 999,999.99') AS avg_deposit_amount
FROM
(
    -- Find the avegrage count and amount of deposits made by each customer
    SELECT 
        customer_id, 
        COUNT(txn_type) as deposit_count, 
        AVG(txn_amount) AS avg_deposit_amount
    FROM data_bank.customer_transactions
    WHERE txn_type = 'deposit'
    GROUP BY customer_id
    ORDER BY customer_id
) d
""", conn)

,nb_deposit,avg_deposit_amount
0,5,$ 508.61


**Result**\
The Data Bank customers made an average of 5 deposits, with an average amount of $ 508.61.

___
#### 3. For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

In [18]:
pd.read_sql("""
WITH counting_transactions_cte AS
(
    SELECT 
        customer_id,
        EXTRACT(MONTH FROM txn_date)::INTEGER AS month,
        TO_CHAR(txn_date, 'MONTH') AS month_name,
        SUM(CASE WHEN txn_type = 'deposit' THEN 1 ELSE 0 END) AS deposit,
        SUM(CASE WHEN txn_type = 'purchase' THEN 1 ELSE 0 END) AS purchase,
        SUM(CASE WHEN txn_type = 'withdrawal' THEN 1 ELSE 0 END) AS withdrawal
    FROM data_bank.customer_transactions
    GROUP BY customer_id, month, month_name
    ORDER BY customer_id
)
SELECT 
    month, 
    month_name, 
    COUNT(DISTINCT customer_id) AS nb_customers
FROM counting_transactions_cte
WHERE deposit > 1 AND (purchase > 0 OR withdrawal > 0)
GROUP BY month, month_name
ORDER BY month;
""", conn)

,month,month_name,nb_customers
0,1,JANUARY,168
1,2,FEBRUARY,181
2,3,MARCH,192
3,4,APRIL,70


**Result**
- **March** is the month when Data Bank customers make the most transactions (more than 1 deposit and either 1 purchase or 1 withdrawal in a single month), with a total of 192 customers.
- Following March, **February** is the second month with the highest number of customers engaging in the given transactions.
- In **January**, a total of 168 customers made more than 1 deposit and either 1 purchase or 1 withdrawal.
- The month of **April** has the fewest number of customers conducting these types of transactions, with only 70 clients.

___
#### 4. What is the closing balance for each customer at the end of the month?

Here are the results showing the first 5 customers.

In [19]:
pd.read_sql("""
SELECT 
    customer_id,
    EXTRACT(MONTH FROM txn_date)::INTEGER AS month,
    TO_CHAR(txn_date, 'MONTH') AS month_name,
    SUM(CASE WHEN txn_type = 'deposit' THEN txn_amount ELSE - txn_amount END) AS closing_balance
FROM data_bank.customer_transactions
WHERE customer_id <= 5
GROUP BY customer_id, month, month_name
ORDER BY customer_id, month;
""", conn)

,customer_id,month,month_name,closing_balance
0,1,1,JANUARY,312
1,1,3,MARCH,-952
2,2,1,JANUARY,549
3,2,3,MARCH,61
4,3,1,JANUARY,144
5,3,2,FEBRUARY,-965
6,3,3,MARCH,-401
7,3,4,APRIL,493
8,4,1,JANUARY,848
9,4,3,MARCH,-193


___
#### 5. What is the percentage of customers who increase their closing balance by more than 5%?

In [20]:
pd.read_sql("""
WITH monthly_balance_cte AS
(
    SELECT 
        customer_id,
        EXTRACT(MONTH FROM txn_date)::INTEGER AS month,
        SUM(CASE WHEN txn_type = 'deposit' THEN txn_amount ELSE - txn_amount END) AS closing_balance
    FROM data_bank.customer_transactions
    GROUP BY customer_id, month
    ORDER BY customer_id, month
),
balance_greaterthan5_cte AS
(
    SELECT COUNT(DISTINCT customer_id) AS nb_customers
    FROM
    (
        SELECT 
            customer_id, 
            (LEAD(closing_balance) OVER (PARTITION BY customer_id ORDER BY month) - closing_balance)/ closing_balance::numeric*100 AS percent_change
        FROM monthly_balance_cte
    ) pc
    WHERE percent_change > 5
)
SELECT 
    MAX(nb_customers) AS nb_customers, 
    COUNT(DISTINCT ct.customer_id) AS total_customers,
    CONCAT(ROUND(MAX(nb_customers)/COUNT(DISTINCT ct.customer_id)::numeric * 100,1), ' %') AS percentage_customers
FROM balance_greaterthan5_cte b, data_bank.customer_transactions ct
""", conn)

,nb_customers,total_customers,percentage_customers
0,269,500,53.8 %


**Result**\
There are 53.8 % of the Data Bank customers who increase their closing balance by more than 5%.

___
<a id = 'C'></a>
## C. Data Allocation Challenge

To test out a few different hypotheses - the Data Bank team wants to run an experiment where different groups of customers would be allocated data using 3 different options:

- `Option 1`: data is allocated based off the amount of money at the end of the previous month
- `Option 2`: data is allocated on the average amount of money kept in the account in the previous 30 days
- `Option 3`: data is updated real-time

For this multi-part challenge question - you have been requested to generate the following data elements to help the Data Bank team estimate how much data will need to be provisioned for each option:
- running customer balance column that includes the impact each transaction
- customer balance at the end of each month
- minimum, average and maximum values of the running balance for each customer

Using all of the data available - how much data would have been required for each option on a monthly basis?

### Running Balance

In [21]:
pd.read_sql("""
SELECT *, 
    SUM(
        CASE WHEN txn_type = 'deposit' THEN txn_amount ELSE -txn_amount END
    ) OVER (PARTITION BY customer_id ORDER BY txn_date) AS running_balance
FROM data_bank.customer_transactions
WHERE customer_id <= 5;
""", conn)

,customer_id,txn_date,txn_type,txn_amount,running_balance
0,1,2020-01-02,deposit,312,312
1,1,2020-03-05,purchase,612,-300
2,1,2020-03-17,deposit,324,24
3,1,2020-03-19,purchase,664,-640
4,2,2020-01-03,deposit,549,549
5,2,2020-03-24,deposit,61,610
6,3,2020-01-27,deposit,144,144
7,3,2020-02-22,purchase,965,-821
8,3,2020-03-05,withdrawal,213,-1034
9,3,2020-03-19,withdrawal,188,-1222


### Monthly Balance

In [22]:
pd.read_sql("""
SELECT 
    customer_id,
    EXTRACT(MONTH FROM txn_date)::INTEGER AS month,
    TO_CHAR(txn_date, 'Month') AS month_name,
    SUM(CASE WHEN txn_type = 'deposit' THEN txn_amount ELSE -txn_amount END) AS closing_balance
FROM data_bank.customer_transactions
WHERE customer_id <= 5
GROUP BY customer_id, month, month_name
ORDER BY customer_id, month;
""", conn)

,customer_id,month,month_name,closing_balance
0,1,1,January,312
1,1,3,March,-952
2,2,1,January,549
3,2,3,March,61
4,3,1,January,144
5,3,2,February,-965
6,3,3,March,-401
7,3,4,April,493
8,4,1,January,848
9,4,3,March,-193


### Min, Average, Max Transaction

In [23]:
pd.read_sql("""
SELECT 
    customer_id, 
    MIN(running_balance) AS min_transaction, 
    MAX(running_balance) AS max_transaction,
    ROUND(AVG(running_balance),2) AS avg_transaction
FROM 
(
    SELECT *,
        SUM(CASE WHEN txn_type = 'deposit' THEN txn_amount ELSE -txn_amount END) OVER (PARTITION BY customer_id ORDER BY txn_date) AS running_balance
    FROM data_bank.customer_transactions
) running_balance
WHERE customer_id <= 10
GROUP BY customer_id
ORDER BY customer_id
""", conn)

,customer_id,min_transaction,max_transaction,avg_transaction
0,1,-640,312,-151.00
1,2,549,610,579.50
2,3,-1222,144,-732.40
3,4,458,848,653.67
4,5,-2413,1780,-135.45
5,6,-552,2197,624.00
6,7,887,3539,2268.69
7,8,-1029,1363,173.70
8,9,-91,2030,1021.70
9,10,-5090,556,-2229.83


In [24]:
conn.close()